In [44]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gensim
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [45]:
import re
model = gensim.models.Word2Vec.load("data//models//w2v_model_w9m30.model")
with open('data//pairs_gender.txt' , encoding = 'utf-8') as f:
    pairs = f.readlines()

dif = np.zeros(300)
for line in pairs:
    pair = re.sub("[^а-я ]", "", line).split(' ')
    print(pair)
    positive_word = pair[0]
    negative_word = pair[1]
    dif = dif + model.wv[model.wv.key_to_index[positive_word] ] - model.wv[model.wv.key_to_index[negative_word]]

['мужчина', 'женщина']
['он', 'она']
['парень', 'девушка']
['мальчик', 'девочка']
['отец', 'мать']
['мужской', 'женский']


In [46]:
def ranking_score(model, word, dif):
    word = model.wv[model.wv.key_to_index[word]]
    projection_value = np.dot(word, dif) /( np.linalg.norm(dif))
    return projection_value

def ranking_score_cosine(model, word, dif):
    word = model.wv[model.wv.key_to_index[word]]
    projection_value = np.dot(word, dif) /( np.linalg.norm(dif)*np.linalg.norm(word))
    return projection_value

In [47]:
def normalize_words(words_list):
    norm_list = []
    for word in words_list:
        p = morph.parse(word.lower())[0]
        norm_list.append(p.normal_form)
    return norm_list

In [48]:
def words_rank(model, words_list, dif):
    ranking_res = []
    for word in words_list:
        score = ranking_score(model, word, dif)
        ranking_res.append((word, score))
    return sorted(ranking_res, key=lambda x: x[1])

In [49]:
with open('data//words_to_rank.txt' , encoding = 'ANSI') as f:
    words_list = f.readlines()[0].split(' ')
print(words_list)

['материнство', 'женщина', 'девочка', 'нумерология', 'эзотерика', 'астролог', 'психолог', 'психотерапия', 'платье', 'таролог', 'астрология', 'кухня', 'таро', 'психология', 'бизнес', 'миллионер', 'заработок', 'богатство', 'богатый', 'деньга', 'футбол', 'баскетбол', 'зарплата', 'пожарный', 'мужчина', 'психотерапевт', 'мальчик']


In [50]:
norm_words = normalize_words(words_list)
words_rank(model, norm_words, dif)

[('материнство', -0.9669663206474277),
 ('женщина', -0.46714358039440207),
 ('девочка', -0.4443107050381663),
 ('таролог', -0.37463429148574573),
 ('нумерология', -0.32061087914278796),
 ('астролог', -0.3067263163253954),
 ('таро', -0.2611251304267806),
 ('платье', -0.25193589709554154),
 ('астрология', -0.24368514284186213),
 ('психолог', -0.21524620856238216),
 ('эзотерика', -0.2086764764696825),
 ('бизнес', -0.10458448249145634),
 ('кухня', -0.028796285044676895),
 ('психология', -0.019679674749069814),
 ('заработок', 0.017778002352894083),
 ('психотерапия', 0.017905874135632265),
 ('психотерапевт', 0.10585297999540388),
 ('богатство', 0.11050171708163187),
 ('деньга', 0.11069552293730156),
 ('богатый', 0.21835992244331037),
 ('зарплата', 0.23999112320361504),
 ('футбол', 0.28544291703301894),
 ('миллионер', 0.3106327777704206),
 ('мужчина', 0.4992537778390033),
 ('баскетбол', 0.5164509258986895),
 ('пожарный', 0.7669169178268989),
 ('мальчик', 1.1549872692919922)]